In [ ]:
import numpy as np 
import pandas as pd 
import re 

In [ ]:
df = pd.read_csv('./data/BankFAQs.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['Class'].value_counts()

In [ ]:
questions = df['Question']
answers = df['Answer']

In [ ]:
questions

In [ ]:
# !pip install contractions

In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk 
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
import contractions

In [ ]:
lemmatizer = WordNetLemmatizer()
def clean_text(text):
  text = text.lower()
  text = BeautifulSoup(text,'lxml').text
  text = contractions.fix(text)
  text = re.sub(r'[^A-Za-z\s]','',text)
  
  text = re.sub(r'http\S+|https\S+|www\S+','',text, flags=re.MULTILINE)
  text = re.sub(r'\S+@\S+','',text)
  text = re.sub('\s+','  ',text).strip()
  words = word_tokenize(text)
  words = [lemmatizer.lemmatize(w)  for w in words if w not in stop_words]
  text = ' '.join(words)
  return text  

In [ ]:
questions = [clean_text(i) for i in questions]
answers = [clean_text(i) for i in  answers]

In [ ]:
word2count = {}
for line in questions:
  for word in line.split():
    if word not in word2count:
      word2count[word] = 1 
    else:
      word2count[word] +=1 
      
for line in answers:
  for word in line.split():
    if word not in word2count:
      word2count[word] = 1 
    else:
      word2count[word] +=1 
    

In [ ]:
word2count

In [ ]:
vocab = {}
thresh = 5 
a = 0
for val,count in word2count.items():
  if count >=5:
    vocab[val] = a
    a +=1 


In [ ]:
vocab

In [ ]:
max_in_len = max(len(w.split()) for w in questions)
max_tr_len = max(len(w.split()) for w in answers)

print(max_in_len)
print(max_tr_len)

In [ ]:
for i in range(len(answers)):
    answers[i] = ' '.join(answers[i].split()[:14])

In [ ]:
max_in_len = max(len(w.split()) for w in questions)
max_tr_len = max(len(w.split()) for w in answers)

print(max_in_len)
print(max_tr_len)

In [ ]:
a = 1/0
a

In [ ]:
for i in range(len(answers)):
  answers[i] = '<SOS> ' + answers[i] + ' <EOS>' 
answers

In [ ]:
max_in_len = max(len(w.split()) for w in questions)
max_tr_len = max(len(w.split()) for w in answers)

print(max_in_len)
print(max_tr_len)

In [ ]:
tokens = ['<PAD>','<SOS>','<OUT>','<EOS>']
x = len(vocab)
for token in tokens:
  vocab[token] = x 
  x +=1

In [ ]:
vocab['document'] = vocab['<PAD>']
vocab['<PAD>'] = 0

In [ ]:
inv_vocab = {w:v for v,w in vocab.items()}
inv_vocab

In [ ]:
encoder_inp = []
for i in questions:
  lst = []
  for w in i.split():
    if w not in vocab:
      lst.append(vocab['<OUT>'])
    else:
      lst.append(vocab[w])
  encoder_inp.append(lst)
      
decoder_inp = []
for i in answers:
  lst = []
  for w in i.split():
    if w not in vocab:
      lst.append(vocab['<OUT>'])
    else:
      lst.append(vocab[w])
      
  decoder_inp.append(lst)

In [ ]:
encoder_inp

In [ ]:
# encoder_inp
decoder_inp

In [ ]:
from keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp,max_in_len,padding='post',truncating='post')
decoder_inp = pad_sequences(decoder_inp,max_tr_len,padding='post',truncating='post')

In [ ]:
encoder_inp.shape
# decoder_inp.shape

In [ ]:
# -----------remove sos from decoder inp----------------------------
# Decoder Input: It is the input sequence fed into the decoder at each time step to predict the next token.

# To align the decoder input with the target sequence for training, the <sos> token is included at the beginning of the decoder input, but it is not needed in the decoder target sequence because the target sequence is supposed to predict the next tokens without including the <sos> itself.
decoder_final_output = []
for i in decoder_inp:
  decoder_final_output.append(i[1:])

In [ ]:
decoder_final_output

In [ ]:
decoder_final_output = pad_sequences(decoder_final_output,max_tr_len,padding='post',truncating='post')

In [ ]:
decoder_final_output.shape

In [ ]:
decoder_inp.shape

In [ ]:
from keras.utils import to_categorical 
decoder_final_output = to_categorical(decoder_final_output,len(vocab))

In [ ]:
decoder_final_output

In [ ]:
decoder_final_output.shape

In [ ]:
# ------------explantion of blw model  ----------------------

# Step-by-Step Explanation
# Input Layer for Encoder and Decoder:

# python
# Copy code
# enc_inp = Input(shape=(13,))
# dec_inp = Input(shape=(13,))
# enc_inp and dec_inp are placeholders for input sequences for the encoder and decoder, respectively. Both have a fixed length of 13 tokens.
# Embedding Layer:

# python
# Copy code
# VOCAB_SIZE = len(vocab)
# embed = Embedding(VOCAB_SIZE+1, output_dim=50, 
#                   input_length=13,
#                   trainable=True)
# VOCAB_SIZE is the number of unique words in your vocabulary.
# Embedding layer transforms input tokens into dense vectors of fixed size (50 in this case). It adds 1 to the vocabulary size to account for special tokens.
# input_length=13 means each input sequence has 13 tokens.
# Encoder Embedding:

# python
# Copy code
# enc_embed = embed(enc_inp)
# The encoder input enc_inp is passed through the embedding layer, converting token sequences into sequences of vectors.
# Encoder LSTM:

# python
# Copy code
# enc_lstm = LSTM(400, return_sequences=True, return_state=True)
# enc_op, h, c = enc_lstm(enc_embed)
# enc_states = [h, c]
# enc_lstm is an LSTM layer with 400 units.
# return_sequences=True means it returns the entire output sequence.
# return_state=True means it returns the last hidden state (h) and cell state (c).
# enc_op is the output sequence from the LSTM.
# h and c are the final hidden and cell states, respectively.
# enc_states = [h, c] will be used to initialize the decoder's LSTM.
# Decoder Embedding:

# python
# Copy code
# dec_embed = embed(dec_inp)
# The decoder input dec_inp is also passed through the same embedding layer, converting token sequences into sequences of vectors.
# Decoder LSTM:

# python
# Copy code
# dec_lstm = LSTM(400, return_sequences=True, return_state=True)
# dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)
# dec_lstm is another LSTM layer with 400 units.
# dec_op is the output sequence from the LSTM.
# The decoder's LSTM is initialized with the encoder's final states (enc_states).
# Dense Layer:

# python
# Copy code
# dense = Dense(VOCAB_SIZE, activation='softmax')
# Dense layer with VOCAB_SIZE units and softmax activation to predict the next word in the sequence for each time step.
# Output from Dense Layer:

# python
# Copy code
# dense_op = dense(dec_op)
# The output of the decoder LSTM (dec_op) is passed through the dense layer to get the final predictions.
# Define the Model:

# python
# Copy code
# model = Model([enc_inp, dec_inp], dense_op)
# Define the model with inputs as [enc_inp, dec_inp] and output as dense_op.
# Compile the Model:

# python
# Copy code
# model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# Compile the model with categorical cross-entropy loss, accuracy metric, and Adam optimizer.
# Train the Model:

# python
# Copy code
# model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=15)
# Train the model on the training data for 15 epochs. encoder_inp and decoder_inp are the input sequences, and decoder_final_output is the expected output sequences for training.

In [ ]:
from keras.layers import Input,Embedding,LSTM,Dense
from keras.models import Model 
VOCAB_SIZE = len(vocab)
enc_inp = Input(shape=(16,))
dec_inp = Input(shape=(16,))


embed = Embedding(VOCAB_SIZE+1,output_dim = 50,input_length=16,trainable=True)
enc_embed = embed(enc_inp)
enc_lstm = LSTM(400,return_sequences=True,return_state=True)
enc_op,h,c = enc_lstm(enc_embed)

en_states = [h,c]

dec_embed = embed(dec_inp)
dec_lstm = LSTM(400,return_sequences=True,return_state=True)
dec_op,_,_ = dec_lstm(dec_embed,initial_state=en_states)

dense = Dense(VOCAB_SIZE,activation='softmax')
dense_op = dense(dec_op)

model = Model([enc_inp,dec_inp],dense_op)

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=70)


Inference model

In [ ]:
from keras.models import Model
from keras.layers import Input 

enc_model = Model([enc_inp],en_states)

decoder_state_input_h = Input(shape=(400,))
decoder_state_input_c = Input(shape=(400,))
decoder_state_initial_input = [decoder_state_input_h,decoder_state_input_c]

decoder_outputs,state_h,state_c = dec_lstm(dec_embed,initial_state=decoder_state_initial_input)

decoder_states = [state_h,state_c]


dec_model = Model([dec_inp]+decoder_state_initial_input,[decoder_outputs]+decoder_states)


In [ ]:
import numpy as np


from keras.preprocessing.sequence import pad_sequences
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    prepro1  = input("you : ")
    ## prepro1 = "Hello"

    prepro1 = clean_text(prepro1)
    ## prepro1 = "hello"

    prepro = [prepro1]
    ## prepro1 = ["hello"]

    txt = []
    for x in prepro:
        # x = "hello"
        lst = []
        for y in x.split():
            ## y = "hello"
            try:
                lst.append(vocab[y])
                ## vocab['hello'] = 454
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    ## txt = [[454]]
    txt = pad_sequences(txt, 16, padding='post')

    ## txt = [[454,0,0,0,.........13]]

    stat = enc_model.predict( txt )

    empty_target_seq = np.zeros( ( 1 , 1) )
     ##   empty_target_seq = [0]


    empty_target_seq[0, 0] = vocab['<SOS>']
    ##    empty_target_seq = [255]

    stop_condition = False
    decoded_translation = ''

    while not stop_condition :

        dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
        decoder_concat_input = dense(dec_outputs)
        
        ##decoder_concat_input[0, -1, :] gives [0.1, 0.2, 0.4, 0.3, ...], which are the probabilities for each word in the vocabulary at the current time step.

        sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
        ## sampled_word_index = [2]

        sampled_word = inv_vocab[sampled_word_index] + ' '

        ## inv_vocab[2] = 'hi'
        ## sampled_word = 'hi '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 16:
            stop_condition = True 

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        ## <SOS> - > hi
        ## hi --> <EOS>
        stat = [h, c]  

    print("chatbot attention : ", decoded_translation )
    print("==============================================")  

In [ ]:
# questions
answers